In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from AIBaseClass import ChessAI

# Aufgabe 02: Minimax (einfacher Materialwert)

Die Funktion `get_next_middle_game_move` nimmt das aktuelle Board als Argument und gibt einen super ausgewählten Zug aus der Menge aller gültigen Züge zurück.   
Alle restlichen Funktionen werden von der Basisklasse `ChessAI` vererbt. 

In [ ]:
import chess

class Exercise02AI(ChessAI):
    """Chooses middle game moves using minimax algorithm and material values"""
    
    LIMIT = 99999

    def __init__(self, player_name: str, search_depth: int = 2) -> None:
        super().__init__(player_name)
        self.DEPTH = search_depth

    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move | None:
        '''Picks a cool move out of all legal board moves'''
        moves = list(board.legal_moves)
        if not moves:
            return None
        return self.best_move(board)

## Evaluierungsfunktion

In [ ]:
class Exercise02AI(Exercise02AI): # type: ignore
    MATERIAL_VALUE_PAWN = 1
    MATERIAL_VALUE_KNIGHT = 3
    MATERIAL_VALUE_BISHOP = 3
    MATERIAL_VALUE_ROOK = 5
    MATERIAL_VALUE_QUEEN = 9

    def evaluate(self, board: chess.Board) -> int:
        """Evaluates a given board. 
        Returns a positive value if white has a better material value than black."""
        # Get the amount of pieces on the board per color 
        amount_pawn_white   = len(board.pieces(chess.PAWN,   chess.WHITE))
        amount_pawn_black   = len(board.pieces(chess.PAWN,   chess.BLACK))
        amount_knight_white = len(board.pieces(chess.KNIGHT, chess.WHITE))
        amount_knight_black = len(board.pieces(chess.KNIGHT, chess.BLACK))
        amount_bishop_white = len(board.pieces(chess.BISHOP, chess.WHITE))
        amount_bishop_black = len(board.pieces(chess.BISHOP, chess.BLACK))
        amount_rook_white   = len(board.pieces(chess.ROOK,   chess.WHITE))
        amount_rook_black   = len(board.pieces(chess.ROOK,   chess.BLACK))
        amount_queen_white  = len(board.pieces(chess.QUEEN,  chess.WHITE))
        amount_queen_black  = len(board.pieces(chess.QUEEN,  chess.BLACK))
        
        # Get difference
        diff_pawn   = amount_pawn_white   - amount_pawn_black
        diff_knight = amount_knight_white - amount_knight_black
        diff_bishop = amount_bishop_white - amount_bishop_black
        diff_rook   = amount_rook_white   - amount_rook_black
        diff_queen  = amount_queen_white  - amount_queen_black
        # Calculate material value
        return sum([
            self.MATERIAL_VALUE_PAWN * diff_pawn,
            self.MATERIAL_VALUE_KNIGHT * diff_knight,
            self.MATERIAL_VALUE_BISHOP * diff_bishop,
            self.MATERIAL_VALUE_ROOK * diff_rook,
            self.MATERIAL_VALUE_QUEEN * diff_queen
        ])

In [ ]:
board = chess.Board()
board.set_fen("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise02AI("Testplayer", 3)
player.evaluate(board)

## Minimax

In [ ]:
# Debugging start
from copy import deepcopy
# Debugging end
class Exercise02AI(Exercise02AI): # type: ignore
    # Debugging start
    path_history: dict = {}
    # Debugging end
    def minimax(self, board: chess.Board, depth: int) -> int:
        """Searches the best value with given depth using minimax algorithm"""
        if depth == 0 or board.is_checkmate() or board.is_stalemate() or board.is_insufficient_material():
            return self.evaluate(board)
        

        if board.turn:
            maxEvaluation = -self.LIMIT
            for move in board.legal_moves:
                board.push(move)
                evaluation = self.minimax(board, depth - 1)
                # Debugging start
                if maxEvaluation < evaluation:
                    board_copy = deepcopy(board)
                    board_copy.pop()
                    Exercise02AI.path_history[str(board_copy) + str(depth)] = move
                # Debugging end
                board.pop()
                maxEvaluation = max(maxEvaluation, evaluation)
            return maxEvaluation
        else:
            minEvaluation = self.LIMIT
            for move in board.legal_moves:
                board.push(move)
                evaluation = self.minimax(board, depth - 1)
                # Debugging start
                if minEvaluation > evaluation:
                    board_copy = deepcopy(board)
                    board_copy.pop()
                    Exercise02AI.path_history[str(board_copy) + str(depth)] = move
                # Debugging end
                board.pop()
                minEvaluation = min(minEvaluation, evaluation)
            return minEvaluation

## Berechnung des besten Zuges

In [ ]:
class Exercise02AI(Exercise02AI): # type: ignore
    def best_move(self, board: chess.Board):
        """Gets the best next move"""
        # Debugging start
        Exercise02AI.path_history = {}
        # Debugging end
        best_move_val = -self.LIMIT if board.turn else self.LIMIT
        final_move = None

        for move in board.legal_moves:
            board.push(move)
            move_val = self.minimax(board, self.DEPTH)
            if (best_move_val < move_val) != board.turn:
                best_move_val = move_val
                final_move = move
            board.pop()
        return final_move

In [ ]:
# Debugging function
from IPython.display import display
def display_path(board_orig, move, depth):
    board = deepcopy(board_orig)
    display(board)
    board.push(move)
    display(board)
    for i in range(depth, 0, -1):
        move = Exercise02AI.path_history[str(board) + str(i)]
        board.push(move)
        display(board)

In [ ]:
DEPTH = 3
board = chess.Board()
board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise02AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.best_move(board)
print(move)
display_path(board, move, DEPTH)